In [1]:
# File name: Auto FPS Reduce ML.ipynb
# Author: Sai Vishwanath V , Aditya Prem
# Date created: 24/9/2019
# Date last modified: 26/9/2019 (Sai)
# Python Version: 3

### In this notebook we 

1. Automate result collection for several reduction factors. We accordingly change the bag sizes in accordance to the specifications mentioned in the paper such as <br>
#####bagsize = bagsize/reduction_factor<br>

In [1]:
import make_hof_half
from make_hof_half import get_processed_hof
import pandas as pd
import os,datetime
import matplotlib.pyplot as plt 
import numpy as np
import cv2
import pathlib
import pickle as pkl

In [2]:
## Get hof , with reduced fps

In [3]:
from video_stuff import get_frames_per
from video_stuff import hof,getFlow
from time import time 
# from make_emi_data import emirnn_preprocess

In [4]:
# EMI imports
from rnn import EMI_DataPipeline
from rnn import EMI_BasicLSTM, EMI_FastGRNN, EMI_FastRNN, EMI_GRU
from emirnnTrainer import EMI_Trainer, EMI_Driver
import utils
import os 
import numpy as np
import tensorflow as tf

/home/adithyapa4444_gmail_com/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/adithyapa4444_gmail_com/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/adithyapa4444_gmail_com/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/adithyapa4444_gmail_com/.local/lib

# Time to automate

In [5]:
from metrics import emi_model_metrics
import numpy as np
import matplotlib

In [6]:
who

EMI_BasicLSTM	 EMI_DataPipeline	 EMI_Driver	 EMI_FastGRNN	 EMI_FastRNN	 EMI_GRU	 EMI_Trainer	 cv2	 datetime	 
emi_model_metrics	 getFlow	 get_frames_per	 get_processed_hof	 hof	 make_hof_half	 matplotlib	 np	 os	 
pathlib	 pd	 pkl	 plt	 tf	 time	 utils	 


In [8]:
def get_hof(videopath,reduction_factor = 2):
    cap = cv2.VideoCapt
    
    ure(videopath)
    fps_counter = 0
    feat_sets = []
    ret, frame_prev = cap.read()
    if not ret:
        return
    frame_prev = cv2.cvtColor(frame_prev, cv2.COLOR_BGR2GRAY)
    frame_prev = cv2.resize(frame_prev, (120,160))
    while cap.isOpened():
        if (fps_counter%reduction_factor==0): #Reducing the fps by n (Reading every nth frame)
            ret, frame_next = cap.read()
            if not ret:
                break
            frame_next = cv2.cvtColor(frame_next, cv2.COLOR_BGR2GRAY)
            frame_next = cv2.resize(frame_next, (120,160))
            feat_sets.append(hof(flow = getFlow(frame_prev, frame_next), orientations = 5, cells_per_block = (6,6), pixels_per_cell = (20,20)))
            frame_prev = frame_next.copy()
        else:
            _,_ = cap.read()
            
            
            fps_counter+=1
    feat_sets = np.asarray(feat_sets)
    col_names = list('feat_' + str(i) for i in range(feat_sets.shape[-1]))
    #hog_feats = np.asarray(hog_feats)
    tuple_feats = list(map(lambda x: tuple(feat_sets[x]), range(len(feat_sets))))
    df = pd.DataFrame(tuple_feats, columns = col_names)
    df['video_path'] = videopath
    return df

In [35]:
reduction_factors = [6]
bagsizes = [4]
subinst = [2]
stride = [1]


In [ ]:
for rf, bag, si, st in zip(reduction_factors,bagsizes, subinst, stride):
    file_1 = open('/home/adithyapa4444_gmail_com/FPS_Expt_Results/get_hof_times.txt','a')
    start = time()
    hof_df = get_processed_hof(src_folder='/home/adithyapa4444_gmail_com/UCF_Crime/Full_UCrime/annotated/',bagsize=bag,reduction_factor=rf)
    time_string = "Bagsize : "+str(bag)+"\nReduction Factor : "+str(rf)+"\nTime :" + str(time()-start) + "\n\n"
    file_1.write(time_string)
    file_1.close()

In [12]:
# Bag_Size =64
No_of_features =540
ExtractedDir = '/home/adithyapa4444_gmail_com/UCF_Crime/half/'
NumClass = 2
# SubinstanceLen = 32
# SubinstanceStride = 8
# Data_csv = '/home/adithyapa4444_gmail_com/UCF_Crime/Crime_HalfFPS.csv'

In [10]:
df = pd.read_csv('/home/adithyapa4444_gmail_com/UCF_Crime/Crime_128_UB.csv')

In [ ]:

x_train, y_train, x_val, y_val, x_test, y_test = emirnn_preprocess(bagsize=128,
                      no_of_features=No_of_features,
                      extractedDir=ExtractedDir,
                      numClass=8,
                      subinstanceLen=64,
                      subinstanceStride=16,
                      data_csv=df,
                      raw_create=1
                     )

### Automation begins here ;)

In [36]:
import os,datetime
from time import time
for rf, bag, si, st in zip(reduction_factors,bagsizes, subinst, stride):
    file_1 = open('/home/adithyapa4444_gmail_com/FPS_Expt_Results/get_hof_times.txt','a')
    start = time()
    hof_df = get_processed_hof(src_folder='/home/adithyapa4444_gmail_com/Peliculas/dataset/',bagsize=bag,reduction_factor=rf)
    time_string = "Bagsize : "+str(bag)+"\nReduction Factor : "+str(rf)+"\nTime :" + str(time()-start) + "\nNumber of bags:" + str(len(hof_df) // bag)
    file_1.write(time_string)
    file_1.close()
    df_copy = hof_df
    x_train, y_train, x_val, y_val, x_test, y_test = emirnn_preprocess(bagsize=bag,
                      no_of_features=540,
                      extractedDir=ExtractedDir,
                      numClass=2,
                      subinstanceLen=si,
                      subinstanceStride=st,
                      data_csv=hof_df,
                      raw_create=1
                     )
    
    data = {
        'x_train' : x_train, 
        'y_train' : y_train, 
        'x_val' : x_val, 
        'y_val' : y_val, 
        'x_test' : x_test, 
        'y_test' : y_test
    }
    
    file_2 = open('/home/adithyapa4444_gmail_com/FPS_Expt_Results/reduced_FPS_metrics.txt','a')
    ## lstm
    dataset = 'CRIME_FULL'
    path = ExtractedDir+'64_16/'
    ''
    #Choose model from among [lstm, fastgrnn, gru]
    model = 'lstm'
    model_prefix = '/home/adithyapa4444_gmail_com/FPS_Expt_Results/' + model + 'p3_' + str(rf) + '/'
    os.mkdir(model_prefix)
    # Dictionary to set the parameters.
    params = {
        "NUM_HIDDEN" : 128,
        "NUM_TIMESTEPS" : int(si), #subinstance length.
        "ORIGINAL_NUM_TIMESTEPS" : int(bag), # Window length 
        "NUM_FEATS" : 540,
        "FORGET_BIAS" : 1.0,
        "NUM_OUTPUT" : 2,   # Number of target classes
        "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
        "KEEP_PROB" : 0.75,
        "PREFETCH_NUM" : 5,
        "BATCH_SIZE" : 32,
        "NUM_EPOCHS" : 2,
        "NUM_ITER" : 3,
        "NUM_ROUNDS" : 4,
        "LEARNING_RATE" : 0.001,
        "MODEL_PREFIX" : model_prefix + 'model-' + str(model)
    }

    #Preprocess data, and load the train,test and validation splits.
    lstm_dict = lstm_experiment_generator(params, data, path)
    metrics = emi_model_metrics(model_prefix + 'model-' + str(model),data['x_test'], data['y_test'], model_prefix, 1010)
    file_2.write('LSTM_skipped_frame_' + str(rf) + '_' + str(bag) + '\n')
    for metric in metrics:
        lstm_push = metric + ' : ' + str(metrics[metric]) + '\n'
        file_2.write(lstm_push)
    ## gru
    dataset = 'CRIME_FULL_GRU'
    path = ExtractedDir+'64_16/'

    #Choose model from among [lstm, fastgrnn, gru]
    model = 'gru'
    model_prefix = '/home/adithyapa4444_gmail_com/FPS_Expt_Results/' + model + 'p2_' + str(rf) + '/'
    os.mkdir(model_prefix)
    # Dictionary to set the parameters.
    gru_params = {
        "NUM_HIDDEN" : 128,
        "NUM_TIMESTEPS" : si,
        "ORIGINAL_NUM_TIMESTEPS" : bag,
        "NUM_FEATS" : 540,
        "FORGET_BIAS" : 1.0,
        "NUM_OUTPUT" : 2,
        "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
        "KEEP_PROB" : 0.75,
        "PREFETCH_NUM" : 5,
        "BATCH_SIZE" : 32,
        "NUM_EPOCHS" : 2,
        "NUM_ITER" : 3,
        "NUM_ROUNDS" : 4,
        "LEARNING_RATE" : 0.001,
        "MODEL_PREFIX" : model_prefix + 'model-' + str(model)
    }
    #Preprocess data, and load the train,test and validation splits.
    gru_dict = gru_experiment_generator(gru_params, data, path)
    metrics = emi_model_metrics(model_prefix + 'model-' + str(model),data['x_test'], data['y_test'], model_prefix, 1010)
    file_2.write('\nGRU_skipped_frame_' + str(rf) + '_' + str(bag) + '\n')
    for metric in metrics:
        lstm_push = metric + ' : ' + str(metrics[metric]) + '\n'
        file_2.write(lstm_push)
    model = 'fastgrnn'
    model_prefix = '/home/adithyapa4444_gmail_com/FPS_Expt_Results/' + model + 'p1_' + str(rf) + '/'
    os.mkdir(model_prefix)
    ## fastgrnn
    fastgrnn_param = {
        "NUM_HIDDEN" : 128,
        "NUM_TIMESTEPS" : si,
        "NUM_FEATS" : 540,
        "FORGET_BIAS" : 1.0,
        "NUM_OUTPUT" : 2,
        "USE_DROPOUT" : 0,

        "KEEP_PROB" : 0.9,
        "UPDATE_NL" : "quantTanh",
        "GATE_NL" : "quantSigm",
        "WRANK" : 5,
        "URANK" : 6,
        "PREFETCH_NUM" : 5,
        "BATCH_SIZE" : 32,
        "NUM_EPOCHS" : 2,
        "NUM_ITER" : 3,
        "NUM_ROUNDS" : 4,
        "MODEL_PREFIX" : model_prefix + 'model-' + str(model)
    }
    fastgrnn_dict = fastgrnn_experiment_generator(fastgrnn_param, data, path)
    metrics = metrics_fastgrnn(model_prefix + 'model-' + str(model), data['x_test'], data['y_test'], 1010)
    file_2.write('FASTGRNN_skipped_frame_' + str(rf) + '_' + str(bag) + '\n')
    for metric in metrics:
        lstm_push = metric + ' : ' + str(metrics[metric]) + '\n'
        file_2.write(lstm_push)
    file_2.write('\n------------------------------------------------------------------------------------------------------\n')
    file_2.close()



(1444, 540)
(1444, 1)
(361, 4, 540)
subinstanceLen: 2 
subinstanceStride : 1 
numClass: 2 
numSteps 4 
numFeats 540
X Shape : (5273, 64, 540)
Y Shape : (5273, 8)
subinstanceLen: 2 
subinstanceStride : 1 
numClass: 2 
numSteps 4 
numFeats 540
X Shape : (259, 4, 540)
Y Shape : (259, 2)
Num train 361
subinstanceLen: 2 
subinstanceStride : 1 
numClass: 2 
numSteps 4 
numFeats 540
X Shape : (73, 4, 540)
Y Shape : (73, 2)
Num test 361
subinstanceLen: 2 
subinstanceStride : 1 
numClass: 2 
numSteps 4 
numFeats 540
X Shape : (29, 4, 540)
Y Shape : (29, 2)
Num val 361
x_train shape is: (259, 3, 2, 540)
y_train shape is: (259, 3, 2)
x_test shape is: (29, 3, 2, 540)
y_test shape is: (29, 3, 2)
Shape of x :  (?, 2, 540)
Length of x :  2
Dimensions of elements of x :  (?, 540)
Update policy: top-k
Training with MI-RNN loss for 2 rounds
Round: 0
Epoch   1 Batch     6 (   15) Loss 0.03814 Acc 0.96875 | Val acc 0.90805 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstmp3_6/model-lstm

INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstmp3_6/model-lstm-1010
x_train shape is: (259, 3, 2, 540)
y_train shape is: (259, 3, 2)
x_test shape is: (29, 3, 2, 540)
y_test shape is: (29, 3, 2)
Update policy: top-k
Training with MI-RNN loss for 2 rounds
Round: 0
Epoch   1 Batch     6 (   15) Loss 0.05044 Acc 0.95833 | Val acc 0.91954 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/grup2_6/model-gru, global_step 1000
Epoch   1 Batch     6 (   15) Loss 0.02587 Acc 0.97917 | Val acc 0.95402 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/grup2_6/model-gru, global_step 1001
Epoch   1 Batch     6 (   15) Loss 0.00449 Acc 1.00000 | Val acc 0.97701 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/grup2_6/model-gru, global_step 1002
INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/grup2_6/model-gru-1002
Round: 1
Epoch   1 Batch     6 (   15) Loss 0.01028 Acc 0.98

INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/grup2_6/model-gru-1010
x_train shape is: (259, 3, 2, 540)
y_train shape is: (259, 3, 2)
x_test shape is: (29, 3, 2, 540)
y_test shape is: (29, 3, 2)
Update policy: top-k
Training with MI-RNN loss for 2 rounds
Round: 0


TypeError: Cannot interpret feed_dict key as Tensor: Can not convert a NoneType into a Tensor.

In [32]:
y_train.shape

(259, 4, 8)

In [3]:
# Cleaner Version
import make_hof_half
from make_hof_half import get_processed_hof
import pandas as pd
import os,datetime
import matplotlib.pyplot as plt 
import numpy as np
import cv2
import pathlib
import pickle as pkl

from video_stuff import get_frames_per
from video_stuff import hof,getFlow
from time import time 
# from make_emi_data import emirnn_preprocess

# EMI imports
from rnn import EMI_DataPipeline
from rnn import EMI_BasicLSTM, EMI_FastGRNN, EMI_FastRNN, EMI_GRU
from emirnnTrainer import EMI_Trainer, EMI_Driver
import utils
import os 
import numpy as np
import tensorflow as tf

from metrics import emi_model_metrics
import numpy as np
import matplotlib

import os,datetime
from time import time

# from emirnn_preprocess import emirnn_preprocess

/home/adithyapa4444_gmail_com/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/adithyapa4444_gmail_com/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/adithyapa4444_gmail_com/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/adithyapa4444_gmail_com/.local/lib

In [4]:
reduction_factors = [3,2]
bagsizes = [43,64]
subinst = [22,32]
stride = [6,8]

In [5]:

    
    
    
    # Bag_Size =64
from time import time
No_of_features =540
# ExtractedDir = '/home/adithyapa4444_gmail_com/UCF_Crime/half/'
NumClass = 8
ExtractedDir = '/home/adithyapa4444_gmail_com/UCF_Crime/half/'
for rf, bag, si, st in zip(reduction_factors,bagsizes, subinst, stride):
    print(rf, bag, si, st)
    
    #--------------------------------------------------------------------
    file_1 = open('/home/adithyapa4444_gmail_com/FPS_Expt_Results/get_hof_times.txt','a')
    #--------------------------------------------------------------------
    start = time()
    hof_df = get_processed_hof(src_folder='/home/adithyapa4444_gmail_com/UCF_Crime/Full_UCrime/annotated/',bagsize=bag,reduction_factor=rf)
    time_string = "\nBagsize : "+str(bag)+"\nReduction Factor : "+str(rf)+"\nTime :" + str(time()-start) + "\nNumber of bags:" + str(len(hof_df) // bag)
    file_1.write(time_string)
    file_1.close()
    x_train, y_train, x_val, y_val, x_test, y_test = emirnn_preprocess(bagsize=bag,
                      no_of_features=No_of_features,
                      extractedDir=ExtractedDir,
                      numClass=8,
                      subinstanceLen=si,
                      subinstanceStride=st,
                      data_csv=hof_df,
                      raw_create=1
                     )
    del hof_df
    #--------------------------------------------------------------
    data = {
        'x_train' : x_train, 
        'y_train' : y_train, 
        'x_val' : x_val, 
        'y_val' : y_val, 
        'x_test' : x_test, 
        'y_test' : y_test
    }
    # -------------------------------------------------------------
    file_2 = open('/home/adithyapa4444_gmail_com/FPS_Expt_Results/reduced_FPS_metrics.txt','a')
    # ----------------------------------------------------
#     print('############FGRNN for reduction factor',rf)
#     dataset = 'CRIME_FULL_FASTGRNN'
#     model = 'fastgrnn'
#     model_prefix = '/home/adithyapa4444_gmail_com/FPS_Expt_Results/' + model + '_' + str(rf) + '/'
#     os.mkdir(model_prefix)
#     path = ExtractedDir+str(si)+'_'+str(st)+'/'
#     ## fastgrnn
#     fastgrnn_param = {
#         "NUM_HIDDEN" : 128,
#         "NUM_TIMESTEPS" : si,
#         "NUM_FEATS" : 540,
#         "FORGET_BIAS" : 1.0,
#         "NUM_OUTPUT" : 8,
#         "USE_DROPOUT" : 0,

#         "KEEP_PROB" : 0.9,
#         "UPDATE_NL" : "quantTanh",
#         "GATE_NL" : "quantSigm",
#         "WRANK" : 5,
#         "URANK" : 6,
#         "PREFETCH_NUM" : 5,
#         "BATCH_SIZE" : 32,
#         "NUM_EPOCHS" : 2,
#         "NUM_ITER" : 3,
#         "NUM_ROUNDS" : 4,
#         "MODEL_PREFIX" : model_prefix + 'model-' + str(model)
#     }
#     fastgrnn_dict = fastgrnn_experiment_generator(fastgrnn_param, data, path)
#     metrics = metrics_fastgrnn(model_prefix + '/model-' + str(model), data['x_test'], data['y_test'], 1010)
#     file_2.write('\nFASTGRNN_skipped_frame_' + str(rf) + '_' + str(bag) + '\n')
#     for metric in metrics:
#         lstm_push = metric + ' : ' + str(metrics[metric]) + '\n'
#         file_2.write(lstm_push)
    #-----------------------------------------------------------------------------------
    ## lstm
    print('############LSTM for reduction factor',rf)
    dataset = 'CRIME_FULL'
    path = ExtractedDir+str(si)+'_'+str(st)+'/'
    #Choose model from among [lstm, fastgrnn, gru]
    model = 'lstm'
    model_prefix = '/home/adithyapa4444_gmail_com/FPS_Expt_Results/' + model + '_' + str(rf) + '/'
    try:
        os.mkdir(model_prefix)
    except:
        print("Chill. Model prefix ",model_prefix," already exists")
    # Dictionary to set the parameters.
    params = {
        "NUM_HIDDEN" : 128,
        "NUM_TIMESTEPS" : int(si), #subinstance length.
        "ORIGINAL_NUM_TIMESTEPS" : int(bag), # Window length 
        "NUM_FEATS" : 540,
        "FORGET_BIAS" : 1.0,
        "NUM_OUTPUT" : 8,   # Number of target classes
        "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
        "KEEP_PROB" : 0.75,
        "PREFETCH_NUM" : 5,
        "BATCH_SIZE" : 32,
        "NUM_EPOCHS" : 2,
        "NUM_ITER" : 3,
        "NUM_ROUNDS" : 4,
        "LEARNING_RATE" : 0.001,
        "MODEL_PREFIX" : model_prefix + 'model-' + str(model)
    }

    #Preprocess data, and load the train,test and validation splits.
    lstm_dict = lstm_experiment_generator(params, data, path)
    metrics = emi_model_metrics(model_prefix + 'model-' + str(model),data['x_test'], data['y_test'], model_prefix, 1010)
    file_2.write('\nLSTM_skipped_frame_' + str(rf) + '_' + str(bag) + '\n')
    for metric in metrics:
        lstm_push = metric + ' : ' + str(metrics[metric]) + '\n'
        file_2.write(lstm_push)
        
    #--------------------------------------------------------------
    print('############gru for reduction factor',rf)
    ## gru
    dataset = 'CRIME_FULL_GRU'
    path = ExtractedDir+str(si)+'_'+str(st)+'/'

    #Choose model from among [lstm, fastgrnn, gru]
    model = 'gru'
    model_prefix = '/home/adithyapa4444_gmail_com/FPS_Expt_Results/' + model + '_' + str(rf) + '/'
    try:
        os.mkdir(model_prefix)
    except:
        print("Chill. Model prefix ",model_prefix," already exists")
    # Dictionary to set the parameters.
    gru_params = {
        "NUM_HIDDEN" : 128,
        "NUM_TIMESTEPS" : si,
        "ORIGINAL_NUM_TIMESTEPS" : bag,
        "NUM_FEATS" : 540,
        "FORGET_BIAS" : 1.0,
        "NUM_OUTPUT" : 8,
        "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
        "KEEP_PROB" : 0.75,
        "PREFETCH_NUM" : 5,
        "BATCH_SIZE" : 32,
        "NUM_EPOCHS" : 2,
        "NUM_ITER" : 3,
        "NUM_ROUNDS" : 4,
        "LEARNING_RATE" : 0.001,
        "MODEL_PREFIX" : model_prefix + 'model-' + str(model)
    }
    #Preprocess data, and load the train,test and validation splits.
    gru_dict = gru_experiment_generator(gru_params, data, path)
    metrics = emi_model_metrics(model_prefix + 'model-' + str(model),data['x_test'], data['y_test'], model_prefix, 1010)
    file_2.write('\nGRU_skipped_frame_' + str(rf) + '_' + str(bag) + '\n')
    for metric in metrics:
        lstm_push = metric + ' : ' + str(metrics[metric]) + '\n'
        file_2.write(lstm_push)
    
    file_2.write('\n------------------------------------------------------------------------------------------------------\n')
    file_2.close()
    #---------------------------------------------------------------------------------

3 43 22 6


/home/adithyapa4444_gmail_com/make_hof_half.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  dfa=pd.concat([dfa,df])


(323919, 540)
(323919, 1)
(7533, 43, 540)
subinstanceLen: 22 
subinstanceStride : 6 
numClass: 8 
numSteps 43 
numFeats 540
X Shape : (5273, 64, 540)
Y Shape : (5273, 8)
subinstanceLen: 22 
subinstanceStride : 6 
numClass: 8 
numSteps 43 
numFeats 540
X Shape : (5423, 43, 540)
Y Shape : (5423, 8)
Num train 7533
subinstanceLen: 22 
subinstanceStride : 6 
numClass: 8 
numSteps 43 
numFeats 540
X Shape : (1507, 43, 540)
Y Shape : (1507, 8)
Num test 7533
subinstanceLen: 22 
subinstanceStride : 6 
numClass: 8 
numSteps 43 
numFeats 540
X Shape : (603, 43, 540)
Y Shape : (603, 8)
Num val 7533
############LSTM for reduction factor 3
x_train shape is: (5423, 5, 22, 540)
y_train shape is: (5423, 5, 8)
x_test shape is: (603, 5, 22, 540)
y_test shape is: (603, 5, 8)


Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator






Update policy: top-k
Training with MI-RNN loss for 2 rounds
Round: 0
Epoch   1 Batch   160 (  330) Loss 0.03082 Acc 0.80625 | Val acc 0.81222 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_3/model-lstm, global_step 1000
Epoch   1 Batch   160 (  330) Loss 0.01586 Acc 0.87500 | Val acc 0.84978 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_3/model-lstm, global_step 1001
Epoch   1 Batch   160 (  330) Loss 0.00915 Acc 0.94375 | Val acc 0.85987 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_3/model-lstm, global_step 1002

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_3/model-lstm-1002
Round: 1
Epoch   1 Batch   160 (  330) Loss 0.00251 Acc 0.98125 | Val acc 0.87153 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_3/model-lstm, global_step 1003
Epoch   1 Batch   160 (  

Round:  2, Validation accuracy: 0.9112, Test Accuracy (k = 2): 0.910418, Additional savings: 0.759353
INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_3/model-lstm-1011
Round:  3, Validation accuracy: 0.9173, Test Accuracy (k = 2): 0.913736, Additional savings: 0.796929
Shape of x :  (?, 22, 540)
Length of x :  22
Dimensions of elements of x :  (?, 540)


...

Update policy: top-k
Training with MI-RNN loss for 2 rounds
Round: 0
Epoch   1 Batch   150 (  315) Loss 0.02219 Acc 0.79375 | Val acc 0.82704 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_2/model-lstm, global_step 1000
Epoch   1 Batch   150 (  315) Loss 0.01173 Acc 0.89375 | Val acc 0.85803 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_2/model-lstm, global_step 1001
Epoch   1 Batch   150 (  315) Loss 0.00423 Acc 0.93750 | Val acc 0.87691 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_2/model-lstm, global_step 1002
INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_2/model-lstm-1002
Round: 1
Epoch   1 Batch   150 (  315) Loss 0.00829 Acc 0.91875 | Val acc 0.87579 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_2/model-lstm, global_step 1003
Epoch   1 Batch   150 (  315) Loss 0.00592 Acc 0.94375 | Val acc 0.89125 | Model saved to /home/adithyapa4444_gmail_co

INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_2/model-lstm-1010
############gru for reduction factor 2
x_train shape is: (5273, 5, 32, 540)
y_train shape is: (5273, 5, 8)
x_test shape is: (586, 5, 32, 540)
y_test shape is: (586, 5, 8)


Update policy: top-k
Training with MI-RNN loss for 2 rounds
Round: 0
Epoch   1 Batch   150 (  315) Loss 0.05391 Acc 0.68125 | Val acc 0.81243 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_2/model-gru, global_step 1000
Epoch   1 Batch   150 (  315) Loss 0.03141 Acc 0.81250 | Val acc 0.84066 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_2/model-gru, global_step 1001
Epoch   1 Batch   150 (  315) Loss 0.01791 Acc 0.90000 | Val acc 0.86513 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_2/model-gru, global_step 1002
INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_2/model-gru-1002
Round: 1
Epoch   1 Batch   150 (  315) Loss 0.01027 Acc 0.93750 | Val acc 0.87441 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_2/model-gru, global_step 1003
Epoch   1 Batch   150 (  315) Loss 0.00459 Acc 0.95000 | Val acc 0.88046 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt

INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_2/model-gru-1009
Accuracy at k = 2: 0.920137
Savings due to MI-RNN : 0.500000
Savings due to Early prediction: 0.836007
Total Savings: 0.918003
   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.912628   0.793427   0.840486   0.769873   0.912628   0.912628   
1    2  0.920137   0.809429   0.846949   0.784074   0.920137   0.920137   
2    3  0.904437   0.788474   0.856856   0.754751   0.904437   0.904437   
3    4  0.868942   0.725816   0.878321   0.675382   0.868942   0.868942   
4    5  0.838908   0.663412   0.899817   0.605405   0.838908   0.838908   

   micro-rec  
0   0.912628  
1   0.920137  
2   0.904437  
3   0.868942  
4   0.838908  
Max accuracy 0.920137 at subsequencelength 2
Max micro-f 0.920137 at subsequencelength 2
Micro-precision 0.920137 at subsequencelength 2
Micro-recall 0.920137 at subsequencelength 2
Max macro-f 0.809429 at subsequencelengt

INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_2/model-gru-1010


In [28]:
# path = ExtractedDir+str(si)+'_'+str(st)+'/' 9381002679
# ## fastgrnn
# fastgrnn_param = {
#     "NUM_HIDDEN" : 128,
#     "NUM_TIMESTEPS" : si,
#     "NUM_FEATS" : 540,
#     "FORGET_BIAS" : 1.0,
#     "NUM_OUTPUT" : 8,
#     "USE_DROPOUT" : 0,

#     "KEEP_PROB" : 0.9,
#     "UPDATE_NL" : "quantTanh",
#     "GATE_NL" : "quantSigm",
#     "WRANK" : 5,
#     "URANK" : 6,
#     "PREFETCH_NUM" : 5,
#     "BATCH_SIZE" : 32,
#     "NUM_EPOCHS" : 2,
#     "NUM_ITER" : 4,
#     "NUM_ROUNDS" : 10,
#     "MODEL_PREFIX" : model_prefix + 'model-' + str(model)
# }
# fastgrnn_dict = fastgrnn_experiment_generator(fastgrnn_param, data, path)
# metrics = metrics_fastgrnn(model_prefix + 'model-' + str(model), data['x_test'], data['y_test'], 1010)
# file_2.write('\nFASTGRNN_skipped_frame_' + str(rf) + '_' + str(bag) + '\n')
# for metric in metrics:
#     lstm_push = metric + ' : ' + str(metrics[metric]) + '\n'
#     file_2.write(lstm_push)
#-----------------------------------------------------------------------------------
## lstm
print('############LSTM for reduction factor',rf)
dataset = 'CRIME_FULL'
path = ExtractedDir+str(si)+'_'+str(st)+'/'
#Choose model from among [lstm, fastgrnn, gru]
model = 'lstm'
model_prefix = '/home/adithyapa4444_gmail_com/FPS_Expt_Results/' + model + '_' + str(rf) + '/'
# os.mkdir(model_prefix)
# Dictionary to set the parameters.
params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : int(si), #subinstance length.
    "ORIGINAL_NUM_TIMESTEPS" : int(bag), # Window length 
    "NUM_FEATS" : 540,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 8,   # Number of target classes
    "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.75,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 2,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 10,
    "LEARNING_RATE" : 0.001,
    "MODEL_PREFIX" : model_prefix + 'model-' + str(model)
}

#Preprocess data, and load the train,test and validation splits.
lstm_dict = lstm_experiment_generator(params, data, path)
metrics = emi_model_metrics(model_prefix + 'model-' + str(model),data['x_test'], data['y_test'], model_prefix, 1010)
file_2.write('\nLSTM_skipped_frame_' + str(rf) + '_' + str(bag) + '\n')
for metric in metrics:
    lstm_push = metric + ' : ' + str(metrics[metric]) + '\n'
    file_2.write(lstm_push)

#--------------------------------------------------------------
print('############gru for reduction factor',rf)
## gru
dataset = 'CRIME_FULL_GRU'
path = ExtractedDir+str(si)+'_'+str(st)+'/'

#Choose model from among [lstm, fastgrnn, gru]
model = 'gru'
model_prefix = '/home/adithyapa4444_gmail_com/FPS_Expt_Results/' + model + '_' + str(rf) + '/'
os.mkdir(model_prefix)
# Dictionary to set the parameters.
gru_params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : si,
    "ORIGINAL_NUM_TIMESTEPS" : bag,
    "NUM_FEATS" : 540,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 8,
    "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.75,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 2,
    "NUM_ITER" : 4]
    
    "NUM_ROUNDS" : 10,
    "LEARNING_RATE" : 0.001,
    "MODEL_PREFIX" : model_prefix + 'model-' + str(model)
}
#Preprocess data, and load the train,test and validation splits.
gru_dict = gru_experiment_generator(gru_params, data, path)
metrics = emi_model_metrics(model_prefix + 'model-' + str(model),data['x_test'], data['y_test'], model_prefix, 1010)
file_2.write('\nGRU_skipped_frame_' + str(rf) + '_' + str(bag) + '\n')
for metric in metrics:
    lstm_push = metric + ' : ' + str(metrics[metric]) + '\n'
    file_2.write(lstm_push)

file_2.write('\n------------------------------------------------------------------------------------------------------\n')
file_2.close()
#---------------------------------------------------------------------------------

############LSTM for reduction factor 5
x_train shape is: (6724, 5, 11, 540)
y_train shape is: (6724, 5, 8)
x_test shape is: (748, 5, 11, 540)
y_test shape is: (748, 5, 8)
Shape of x :  (?, 11, 540)
Length of x :  11
Dimensions of elements of x :  (?, 540)


Update policy: top-k
Training with MI-RNN loss for 2 rounds
Round: 0
Epoch   1 Batch   209 (  420) Loss 0.05806 Acc 0.77500 | Val acc 0.81649 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_5/model-lstm, global_step 1000
Epoch   1 Batch   209 (  420) Loss 0.01912 Acc 0.90000 | Val acc 0.83628 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_5/model-lstm, global_step 1001
Epoch   1 Batch   209 (  420) Loss 0.00471 Acc 0.97500 | Val acc 0.85217 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_5/model-lstm, global_step 1002
INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_5/model-lstm-1002
Round: 1
Epoch   1 Batch   209 (  420) Loss 0.00658 Acc 0.95000 | Val acc 0.86302 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_5/model-lstm, global_step 1003
Epoch   1 Batch   209 (  420) Loss 0.00400 Acc 0.97500 | Val acc 0.87231 | Model saved to /home/adithyapa4444_gmail_co

INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/lstm_5/model-lstm-1010
############gru for reduction factor 5
x_train shape is: (6724, 5, 11, 540)
y_train shape is: (6724, 5, 8)
x_test shape is: (748, 5, 11, 540)
y_test shape is: (748, 5, 8)


Update policy: top-k
Training with MI-RNN loss for 2 rounds
Round: 0
Epoch   1 Batch   209 (  420) Loss 0.05473 Acc 0.85000 | Val acc 0.77257 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_5/model-gru, global_step 1000
Epoch   1 Batch   209 (  420) Loss 0.06746 Acc 0.80000 | Val acc 0.82040 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_5/model-gru, global_step 1001
Epoch   1 Batch   209 (  420) Loss 0.01577 Acc 0.95000 | Val acc 0.84870 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_5/model-gru, global_step 1002
INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_5/model-gru-1002
Round: 1
Epoch   1 Batch   209 (  420) Loss 0.01183 Acc 0.92500 | Val acc 0.85104 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_5/model-gru, global_step 1003
Epoch   1 Batch   209 (  420) Loss 0.00767 Acc 0.97500 | Val acc 0.85347 | Model saved to /home/adithyapa4444_gmail_com/FPS_Expt

INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_5/model-gru-1011
Accuracy at k = 2: 0.903157
Savings due to MI-RNN : 0.476190
Savings due to Early prediction: 0.667532
Total Savings: 0.825850
   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.894061   0.758939   0.829416   0.713576   0.894061   0.894061   
1    2  0.903157   0.775828   0.835007   0.731515   0.903157   0.903157   
2    3  0.890851   0.754955   0.842711   0.706615   0.890851   0.890851   
3    4  0.862493   0.729224   0.894394   0.673407   0.862493   0.862493   
4    5  0.827715   0.691175   0.893399   0.635503   0.827715   0.827715   

   micro-rec  
0   0.894061  
1   0.903157  
2   0.890851  
3   0.862493  
4   0.827715  
Max accuracy 0.903157 at subsequencelength 2
Max micro-f 0.903157 at subsequencelength 2
Micro-precision 0.903157 at subsequencelength 2
Micro-recall 0.903157 at subsequencelength 2
Max macro-f 0.775828 at subsequencelengt

INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/FPS_Expt_Results/gru_5/model-gru-1010


In [ ]:
No_of_features =540
# ExtractedDir = '/home/adithyapa4444_gmail_com/UCF_Crime/half/'
NumClass = 8
ExtractedDir = '/home/adithyapa4444_gmail_com/UCF_Crime/half'
for rf, bag, si, st in zip(reduction_factors,bagsizes, subinst, stride):
    print(rf, bag, si, st)
    
    #--------------------------------------------------------------------
    file_1 = open('/home/adithyapa4444_gmail_com/FPS_Expt_Results/get_hof_times.txt','a')
    #--------------------------------------------------------------------
    print("start feat extract.")
    start = time()
    hof_df = get_processed_hof(src_folder='/home/adithyapa4444_gmail_com/UCF_Crime/Full_UCrime/annotated/',bagsize=bag,reduction_factor=rf)
    time_string = "\nBagsize : "+str(bag)+"\nReduction Factor : "+str(rf)+"\nTime :" + str(time()-start) + "\nNumber of bags:" + str(len(hof_df) // bag)
    file_1.write(time_string)
    file_1.close()
    print(" feat extract. done ... ")
    x_train, y_train, x_val, y_val, x_test, y_test = emirnn_preprocess(bagsize=bag,
                                                                       no_of_features=No_of_features,
                                                                       extractedDir=ExtractedDir,
                                                                       numClass=8,
                                                                      subinstanceLen=si,
                                                                      subinstanceStride=st,
                                                                      data_csv=hof_df,
                                                                      raw_create=1
                                                                     )    

6 26 13 3
start feat extract.


/home/adithyapa4444_gmail_com/make_hof_half.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  dfa=pd.concat([dfa,df])
